# An example tag analysis pipeline

## Importing

In [ ]:
import os

import numpy

from pepars.utils import Illumina_FASTQ_File_Set
from pepars.fileio import fileio
from pepars.analysis import sequencing_reads as sequencing_reads_analysis

from pepars.plotting import plotting
from pepars.plotting import DNA as DNA_plotting
plotting.init_notebook_mode()

from scrap import chromium
from scrap import tags

## Get your data

In [ ]:
# Download example data

EXAMPLE_FILES = {
    "TAG_EXAMPLE_S1_L001_I1_001.fastq.gz": "https://caltech.box.com/shared/static/0844pu7uru75sh114d440ddroed84cwa.gz",
    "TAG_EXAMPLE_S1_L001_R1_001.fastq.gz": "https://caltech.box.com/shared/static/vgamut6dzebb13q0vtag8domi6b49pql.gz",
    "TAG_EXAMPLE_S1_L001_R2_001.fastq.gz": "https://caltech.box.com/shared/static/vn4h9tvswaye32i5oqd304jqywlkg95j.gz"
}

for file_name, remote_file_URL in EXAMPLE_FILES.items():
    
    file_download_path = os.path.join("data", file_name)
    
    fileio.download_remote_file(
        file_URL=remote_file_URL,
        file_path=file_download_path
    )
    
tag_file_name = "LMO.Barcode.Info.csv"

tag_file_path = os.path.join("data", tag_file_name)
tag_file_remote_URL = "https://caltech.box.com/shared/static/bfwabx39pwrg94zri5fuxugo51ed77an.csv"

fileio.download_remote_file(
    file_URL=tag_file_remote_URL,
    file_path=tag_file_path
)

## Put data in a FASTQ File Set

In [ ]:
# To specify a FASTQ file set, give it the directory of where the FASTQ files are contained, and the
# prefix Illumina project name

FASTQ_file_set = Illumina_FASTQ_File_Set("data", "TAG_EXAMPLE")

# Sequencing QC

In [ ]:
# Print out a few of the sequences just to see what they look like

read_index = 0

for sequences in FASTQ_file_set.get_sequence_iterator():
    
    print("Index read: %s" % sequences[0])
    print("Read 1: %s" % sequences[1])
    print("Read 2: %s" % sequences[2])
    
    read_index += 1
    
    if read_index > 5:
        break

for file in FASTQ_file_set.files:
    file.close()
    
FASTQ_file_set.close()

In [ ]:
# Inspect the quality score distribution of all FASTQ files

for file in FASTQ_file_set.files:
    quality_score_distribution = sequencing_reads_analysis.get_quality_score_distribution(file.file_path)
    DNA_plotting.plot_quality_score_distribution(quality_score_distribution, sample_name=file.file_name, interactive=True)

In [ ]:
# Inspect the nucleotide distribution of all FASTQ files

for file in FASTQ_file_set.files:
    nucleotide_distribution = sequencing_reads_analysis.get_nucleotide_distribution(file.file_path)
    DNA_plotting.plot_nucleotide_prevalence_bar_chart(nucleotide_distribution, sample_name=file.file_name, interactive=True)

## Data prep

### Load tags from a file

In [ ]:
tags_from_file = []

with open(tag_file_path) as tag_file:
    for line in tag_file.readlines():
        tags_from_file.append(line.strip().split(",")[-1])
    tags_from_file = tags_from_file[1:]

### Set your cell barcode whitelist

In [ ]:
# In this example, we don't have transcriptome data, so we get the 10x Chromium whitelist
VALID_CELL_BARCODES = chromium.get_chromium_barcodes(chromium_version=3)

## User Parameters

In [ ]:
# Filter sequences out that do not match this index. If you do not have an index sequence, or you have already
# demultiplexed your sample data, you can set this to None
INDEX_SEQUENCE = "ATCACGAT"

# Select which tags are valid for this analysis. You can always filter this out later
VALID_TAGS = tags_from_file

# How close a tag sequence must be to a valid tag. Can be None, 1, or 2
MAXIMUM_TAG_DISTANCE = 1

# How close the index read must be to the index sequence. Can be None, 1, or 2
MAXIMUM_INDEX_DISTANCE = 1

# How close the cell barcode read must be to a valid cell barcode. Can be None, 1, or 2
MAXIMUM_CELL_BARCODE_DISTANCE = 0

# Throw away sequences that have any quality scores less than this. Set to None to disregard
QUALITY_THRESHOLD = None

# Whether or not to collapse UMIs to be considered as a single read
COLLAPSE_UMIS = True

# Whether or not to perform PCR error correction
ERROR_CORRECT = False

## Get cell tag counts

In [ ]:
cell_tag_counts = tags.get_cell_tag_counts(
    FASTQ_file_set,
    valid_cell_barcode_trie=VALID_CELL_BARCODES,
    valid_tags=VALID_TAGS,
    index_sequence=INDEX_SEQUENCE,
    maximum_tag_distance=MAXIMUM_TAG_DISTANCE,
    maximum_index_distance=MAXIMUM_INDEX_DISTANCE,
    maximum_cell_barcode_distance=MAXIMUM_CELL_BARCODE_DISTANCE,
    quality_threshold=QUALITY_THRESHOLD,
    collapse_UMIs=COLLAPSE_UMIS,
    error_correct=ERROR_CORRECT
)

## Inspect the tag results

In [ ]:
cell_tag_counts

In [ ]:
# Plot the counts of each tag
plotting.plot_bar_chart(cell_tag_counts.sum(axis=0), cell_tag_counts.columns, interactive=True)

## Clean up extraneous tags

In [ ]:
# How many tags are expected. We will filter out all but the top tags
NUM_TAGS_EXPECTED = 6
NUM_CELLS_EXPECTED = 5000

In [ ]:
# Filter out tags other than
cell_tag_counts = cell_tag_counts.loc[:, cell_tag_counts.sum(axis=0).sort_values(ascending=False)[0:NUM_TAGS_EXPECTED].index]

In [ ]:
cell_tag_counts

## Inspect tag count distribution

In [ ]:
# Plot a histogram of total tag counts
plotting.plot_histogram(cell_tag_counts.sum(axis=1), num_bins=1000, interactive=True, x_axis_title="Num total tag reads in a cell", y_axis_title="# Cells")

In [ ]:
# Plot a histogram of total tag counts
plotting.plot_histogram(cell_tag_counts.sum(axis=1), num_bins=1000, interactive=True, log_scale=True, x_axis_title="Num tag reads in a cell", y_axis_title="# Cells")

In [ ]:
# Find a desirable cutoff

num_cells_per_cutoff = []
cell_total_tag_counts = cell_tag_counts.sum(axis=1)
cutoffs = numpy.arange(0, cell_total_tag_counts.max().max(), 1)

estimated_tag_count_cutoff = None

for cutoff in cutoffs:
    num_cells = (cell_total_tag_counts > cutoff).sum()
    num_cells_per_cutoff.append(num_cells)
    if not estimated_tag_count_cutoff and num_cells < NUM_CELLS_EXPECTED:
        estimated_tag_count_cutoff = cutoff - 1
    
plotting.plot_scatter(cutoffs, num_cells_per_cutoff, interactive=True, x_axis_title="Cutoff", y_axis_title="# Cells")

print("Assuming %i cells, estimating tag cutoff as %i" % (NUM_CELLS_EXPECTED, estimated_tag_count_cutoff))

In [ ]:
TAG_COUNT_CUTOFF = estimated_tag_count_cutoff

In [ ]:
for tag in cell_tag_counts.columns:
    plotting.plot_histogram(
        cell_tag_counts[cell_tag_counts.sum(axis=1) > TAG_COUNT_CUTOFF][tag],
        interactive=True,
        num_bins=1000,
        log_scale=False,
        title="%s Count Distribution" % tag
    )